## IMPORT & KONFIGURASI

In [22]:
import os
import glob
import whisper
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# --- KONFIGURASI ---
FOLDER_VIDEO = 'Vid'  # Pastikan folder ini berisi video interview bahasa Inggris
MODEL_WHISPER_SIZE = 'base' 

# GANTI 1: Menggunakan model Sentence-BERT khusus Bahasa Inggris
# Model ini lebih ringan dan performanya sangat baik untuk teks Inggris
MODEL_SBERT = 'all-MiniLM-L6-v2' 

# --- DATASET KUNCI JAWABAN (ENGLISH VERSION) ---
kunci_jawaban_db = {
    'interview_question_1': {
        'ideal_answer': "Machine Learning is a subset of artificial intelligence that focuses on building systems that learn from data to improve their performance without being explicitly programmed.",
        'keywords': ['learn from data', 'improve', 'explicitly programmed']
    },
    'interview_question_2': {
        'ideal_answer': "In supervised learning, the algorithm is trained on labeled data, while unsupervised learning deals with unlabeled data to find hidden patterns.",
        'keywords': ['labeled data', 'unlabeled', 'patterns']
    },
    'interview_question_3': {
        'ideal_answer': "In supervised learning, the algorithm is trained on labeled data, while unsupervised learning deals with unlabeled data to find hidden patterns.",
        'keywords': ['labeled data', 'unlabeled', 'patterns']
    },
    # Sesuaikan nama key ini dengan nama file video kamu
}

# --- FUNGSI LOAD MODEL ---
print("Loading English AI Models...")
stt_model = whisper.load_model(MODEL_WHISPER_SIZE)
nlp_model = SentenceTransformer(MODEL_SBERT)
print("Models loaded successfully!\n")

Loading English AI Models...
Models loaded successfully!



## HELPER FUNCTIONS

In [23]:
def analisis_text_detail(transkrip, keywords_wajib):
    """
    Menganalisis Filler Words, Hedging Words (Keraguan), dan Keywords.
    """
    transkrip_lower = transkrip.lower()
    
    # 1. Cek Filler Words (Indikator gugup)
    filler_list = ['um', 'uh', 'like', 'you know', 'actually', 'basically', 'i mean', 'ah', 'er']
    filler_count = 0
    
    for filler in filler_list:
        # Tambah spasi agar tidak mendeteksi kata di dalam kata lain
        count = transkrip_lower.count(f" {filler} ") 
        if count > 0:
            filler_count += count
            
    # 2. Cek Hedging Words (Indikator ketidakyakinan)
    # Kata-kata ini menunjukkan kandidat tidak yakin dengan jawabannya
    hedging_list = ['maybe', 'i think', 'probably', 'sort of', 'kind of', 'i guess', 'perhaps']
    hedging_count = 0
    found_hedging = []
    
    for word in hedging_list:
        if f" {word} " in transkrip_lower:
            count = transkrip_lower.count(f" {word} ")
            hedging_count += count
            found_hedging.append(word)

    # 3. Cek Keywords
    missed_keywords = []
    hit_keywords = []
    
    for key in keywords_wajib:
        if key.lower() in transkrip_lower:
            hit_keywords.append(key)
        else:
            missed_keywords.append(key)
            
    # Hitung persentase keyword
    if len(keywords_wajib) == 0:
        keyword_score = 100
    else:
        keyword_score = (len(hit_keywords) / len(keywords_wajib)) * 100
        
    return {
        'filler_count': filler_count,
        'hedging_count': hedging_count,
        'hedging_words': ", ".join(found_hedging),
        'missed_keywords': missed_keywords,
        'keyword_score': keyword_score
    }

def hitung_confidence_score(wpm, filler_count, hedging_count):
    """
    Menghitung skor kepercayaan diri (0-100) berdasarkan heuristik.
    """
    base_score = 100
    
    # Penalti 1: Filler Words (Dikurangi 2 poin per filler)
    base_score -= (filler_count * 2)
    
    # Penalti 2: Hedging Words (Dikurangi 3 poin per kata ragu)
    base_score -= (hedging_count * 3)
    
    # Penalti 3: Kecepatan Bicara (WPM)
    # Range ideal: 110 - 160 WPM
    if wpm < 100: 
        # Terlalu lambat = penalti (ragu-ragu)
        base_score -= (100 - wpm) * 0.5 
    elif wpm > 170:
        # Terlalu cepat = penalti (gugup/rushed)
        base_score -= (wpm - 170) * 0.5
        
    # Pastikan skor tidak di bawah 0 atau di atas 100
    return max(0, min(base_score, 100))

def generate_automated_feedback(wpm, semantic_score, missed_keywords, confidence_score):
    """
    Membuat feedback otomatis yang lebih cerdas.
    """
    feedback = []
    
    # Feedback Confidence
    if confidence_score >= 85:
        feedback.append("⭐ Terdengar sangat percaya diri dan profesional.")
    elif confidence_score >= 70:
        feedback.append("✅ Penyampaian cukup baik, namun kurangi kata pengisi (filler).")
    else:
        feedback.append("⚠️ Terdengar ragu-ragu. Hindari kata 'umm/uhh' dan 'i think' agar lebih meyakinkan.")
        
    # Feedback Kecepatan
    if wpm < 110:
        feedback.append("(Bicara sedikit lebih cepat agar lebih energik).")
    
    # Feedback Konten
    if len(missed_keywords) > 0:
        feedback.append(f"❌ Poin hilang: {', '.join(missed_keywords[:3])}...")
    
    return " ".join(feedback)

## MAIN PROCESS

In [24]:
def proses_penilaian_interview_lengkap():
    video_paths = glob.glob(os.path.join(FOLDER_VIDEO, '*.webm'))
    video_paths.sort()
    
    laporan_detail = []

    print(f"Starting analysis with CONFIDENCE SCORE for {len(video_paths)} videos...\\n")

    for video_path in video_paths:
        filename = os.path.basename(video_path).replace('.webm', '')
        
        if filename not in kunci_jawaban_db:
            print(f"[SKIP] No answer key for: {filename}")
            continue

        print(f"--> Processing: {filename}...")
        
        # 1. Transcribe
        result = stt_model.transcribe(video_path, language='en') 
        transkrip = result['text'].strip()
        
        # 2. Hitung WPM
        if result['segments']:
            durasi_detik = result['segments'][-1]['end']
        else:
            durasi_detik = 1.0
            
        jumlah_kata = len(transkrip.split())
        wpm = (jumlah_kata / durasi_detik) * 60
        
        # 3. Load Kunci
        data_kunci = kunci_jawaban_db[filename]
        
        # 4. Semantic Score
        emb_kandidat = nlp_model.encode(transkrip, convert_to_tensor=True)
        emb_ideal = nlp_model.encode(data_kunci['ideal_answer'], convert_to_tensor=True)
        semantic_score = util.pytorch_cos_sim(emb_kandidat, emb_ideal).item() * 100
        
        # 5. Analisis Detail (Keywords, Fillers, Hedging)
        analisis = analisis_text_detail(transkrip, data_kunci['keywords'])
        
        # 6. --- BARU: Hitung Confidence Score ---
        confidence_score = hitung_confidence_score(
            wpm, 
            analisis['filler_count'], 
            analisis['hedging_count']
        )
        
        # 7. Final Score (Bobot disesuaikan: Semantic 50%, Keyword 30%, Confidence 20%)
        final_score = (semantic_score * 0.5) + (analisis['keyword_score'] * 0.3) + (confidence_score * 0.2)
        
        # 8. Feedback
        saran = generate_automated_feedback(
            wpm, semantic_score, analisis['missed_keywords'], confidence_score
        )

        laporan_detail.append({
            'Video ID': filename,
            'Transcript': transkrip,
            'WPM': int(wpm),
            'Fillers': analisis['filler_count'],
            'Hedging Words': analisis['hedging_count'], # Jumlah kata ragu
            'Confidence Score': round(confidence_score, 1), # Skor baru
            'Semantic Score': round(semantic_score, 1),
            'Keyword Score': round(analisis['keyword_score'], 1),
            'FINAL SCORE': round(final_score, 1),
            'Feedback': saran
        })
    
    return pd.DataFrame(laporan_detail)

## OUTPUT REPORT

In [25]:
try:
    df_report = proses_penilaian_interview_lengkap()
    
    print("\n=== AI INTERVIEW REPORT (WITH CONFIDENCE SCORE) ===")
    pd.set_option('display.max_colwidth', 100)
    
    # Kolom yang ditampilkan diperbarui
    cols_to_show = ['Video ID', 'FINAL SCORE', 'Confidence Score', 'WPM', 'Fillers', 'Hedging Words', 'Feedback']
    
    display(df_report[cols_to_show])
    
except Exception as e:
    print(f"Error: {e}")

# Menampilkan Video ID dan Transkrip lengkapnya saja
pd.set_option('display.max_colwidth', None) # Agar teks tidak terpotong (...)

# Tampilkan tabel khusus transkrip
print("=== HASIL TRANSKRIP VIDEO ===")
display(df_report[['Video ID', 'Transcript']])

# Jika ingin menyimpan khusus transkrip ke Excel
# df_report[['Video ID', 'Transcript']].to_excel('Transkrip_Only.xlsx', index=False)

Starting analysis with CONFIDENCE SCORE for 5 videos...\n
--> Processing: interview_question_1...


/Users/home/miniconda3/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--> Processing: interview_question_2...


/Users/home/miniconda3/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--> Processing: interview_question_3...


/Users/home/miniconda3/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[SKIP] No answer key for: interview_question_4
[SKIP] No answer key for: interview_question_5

=== AI INTERVIEW REPORT (WITH CONFIDENCE SCORE) ===


,Video ID,FINAL SCORE,Confidence Score,WPM,Fillers,Hedging Words,Feedback
0,interview_question_1,29.1,84.2,86,3,1,"✅ Penyampaian cukup baik, namun kurangi kata pengisi (filler). (Bicara sedikit lebih cepat agar ..."
1,interview_question_2,25.1,77.4,74,5,0,"✅ Penyampaian cukup baik, namun kurangi kata pengisi (filler). (Bicara sedikit lebih cepat agar ..."
2,interview_question_3,28.4,98.0,105,1,0,⭐ Terdengar sangat percaya diri dan profesional. (Bicara sedikit lebih cepat agar lebih energik)...


=== HASIL TRANSKRIP VIDEO ===


,Video ID,Transcript
0,interview_question_1,"Okay, share this specific challenges you faced while walking on the location and how you overcome them. Ah, okay, actually for this challenge just, there are some challenges when I took the starting locations, especially for the project submission that I already working with it. The first one is actually to meet the specific accuracy or population loss for the aggression metrics. And yeah, actually that's just need to take some trial and error with the different architecture. For example, we can try to add more linear, more neurons, changes to the neurons. Or even I also apply the dropout layer. So yeah, it really helps with the the validation loss to become more lower. Right. And yeah, I think that's one of the biggest challenges that I have while walking on these specifications."
1,interview_question_2,"Can you describe your experience with transfer learning and then software how to benefit projects? Ah, okay. About transfer learning is actually we use existing training model from TensorFlow for example like VGC 16, VG G90 and right. Especially for some cases that we need to use GBLerning using Keras applications. For example like image classification we can use transfer learning models which is that's already trained model with exceptionally high accuracy, high performance. Even though it's trained with different data sets but it really helps to improve our model accuracy model loss. For example like mobile net, VGG 19, VGG 16, yeah, efficient net it will help to improve our models comparing to the one if we use traditional CNN model. CNN model with the convolutional 2D, max pulling and yeah it's quite good actually to use transfer learning through the helps with our model performance to improve our model performance."
2,interview_question_3,"Wait, what is this? We scroll the complexity of the form model. You have built and steps you to to ensure it's a QCN. Reference C. Um, um, um, complexity of the form model you have built. And steps you to to ensure it's accuracy. Um, okay, I will take one of my previous project that I use, yeah, I also use the RISUS project for my undergraduate thesis, yeah, for my script C. And I use this model. It's quite challenging, even though it's a very simple project. I use the RISUS project for my undergraduate thesis, yeah, for my script C. And I use this model. It's quite challenging, even though it's a chief high accuracy, yeah, with some dense layer, yeah, with some trow out layer and try to narrow it also with the callback function, yeah, with the new runs. But the problem is the data set is not balanced, yeah. So it has the in-balance class this data sets, yeah, and the the process that I use is to, yeah, to just to use the technique called smooth and, yeah, synthetic of sampling technique with editor-ners neighbor, yeah, which is basically it's just over-sampling and the data sets, yeah. It helps with accuracy."
